In [ ]:
# !pip install pymysql
# !pip install asyncio
# !pip install websockets
# !pip install joblib

In [2]:
from sklearn.preprocessing import MinMaxScaler
import pymysql
from pymysql.cursors import DictCursor
import pandas as pd
import numpy as np
import joblib
from decimal import Decimal
from sklearn.preprocessing import MinMaxScaler
import time
import json
import asyncio
import websockets

In [23]:
# DB 연결
conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')

# 사용자 ID를 입력 받고 사용자의 수조 리스트를 배열에 담기
cursor = conn.cursor()
sql = "select TANK_ID from TANK where COMPANY_ID = 'insogae'" # 로그인시 들어갈 ID
cursor.execute(sql)
rows = cursor.fetchall()
cursor.close()
print(rows)
tankList = []
for row in rows:
    tankList.append(row[0])
model_test = joblib.load('./model/wt11_do_model.pkl')  
# 훈련시킨 모델마다 다 다른결과가 나옴 하지만 큰차이가 없는 결과로 하나만쓰기로 함.
# 차후 6시간짜리를 10분짜리 훈련시킨걸로 예측값 테스트했을때 좋지 않을경우 6시간짜리로 대체할 예정!
tankList

(('WT11',), ('WT12',), ('WT13',), ('WT14',), ('WT21',), ('WT22',), ('WT23',), ('WT31',))
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-12 23:43:22         1750
metadata.json                                  2023-03-12 23:43:22           64
variables.h5                                   2023-03-12 23:43:22       836736
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


['WT11', 'WT12', 'WT13', 'WT14', 'WT21', 'WT22', 'WT23', 'WT31']

In [24]:
# 각 수조의 첫 12행을 제외한 나머지 DELETE 시키기!!

# 데이터베이스 연결
def deleteData():
    cursor = conn.cursor()
    # 각 탱크마다 쿼리를 생성하여 실행
    for tank_name in tankList:
        # 첫 15개의 레코드를 제외한 나머지 레코드 삭제 (LIMIT 15)
        query = '''
        DELETE t1 FROM TANK_DATA_copy t1
        WHERE RECORD_SEQ NOT IN (
            SELECT t2.RECORD_SEQ FROM (
                SELECT RECORD_SEQ FROM TANK_DATA_copy
                WHERE TANK_ID = %s ORDER BY RECORD_DATE ASC LIMIT 2812
            ) AS t2
        )
        AND TANK_ID = %s
        '''
        
        cursor.execute(query, (tank_name, tank_name))

    # 변경 내용을 커밋하고 연결 종료
    cursor.close()
    conn.commit()
    cursor2 = conn.cursor()
    sql = "select count(*) from TANK_DATA_copy"
    cursor2.execute(sql)
    rowCnt = cursor2.fetchall()
    print(rowCnt)
    cursor2.close()

In [25]:
# 12개 행 남기고 다 지우기

deleteData()

((22496,),)


In [26]:
# 예측한 데이터 집어넣기!!  (10분짜리)

conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')
count = 2800; # 4500번째 부터 카운트해서 예측할 예정 즉 TANK_DATA_copy 가 약 36,000 개 정도 예측이 되어야함.

import threading
import time

lock = threading.Lock()
timer = None

def insertData():
    global count
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},13"
        sql = sql.format(tank_id=tank_name, count=count)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    accListSet
    new_list = [[[elem[:-2] for elem in subsublist] for subsublist in sublist] for sublist in data_list_of_tankList]
    for i in range(len(new_list)):
        for j in range(len(new_list[0])):
            new_list[i][j] = new_list[i][j][:12]
    count = count+1
    x = []
    scaler = MinMaxScaler()
    resultDataList = [] # 예측후에 실제값과 예측값 및 측정 시간을 담을 배열 
    for i, tank in enumerate(new_list):
        x.append(tank[0])
        scaler_data = scaler.fit_transform(x[i]) # 독립변수 스케일링
        reshaped_data = scaler_data.reshape(-1,12,4) # shape 변경
        pre = model_test.predict(reshaped_data) # 모델 예측
        reshaped_array = np.tile(pre, (1, 4)) # shape 재설정
        pre_data = scaler.inverse_transform(reshaped_array)  # 실제 예측값 shape = (1,4) // DO, PH, TEMP, SALT
        cursor = conn.cursor()
        sql = f"""insert into TANK_DATA_copy 
                    VALUES (NULL,'{tankList[i]}',{pre_data[0][0]},{pre_data[0][1]},{pre_data[0][2]},{pre_data[0][3]},
                    (SELECT max(record_date)
                    FROM    (SELECT a.*,
                                ROW_NUMBER() OVER (PARTITION BY a.TANK_ID ORDER BY a.RECORD_DATE desc )  AS row_num 
                            from TANK_DATA_copy a ,TANK b
                            where a.TANK_ID = b.TANK_ID
                            and b.COMPANY_ID = 'insogae') sub 
                    where sub.row_num <= 1
                    and sub.TANK_ID = '{tankList[i]}'
                    order by sub.TANK_ID,sub.RECORD_DATE)
                    + INTERVAL 10 MINUTE)"""
        cursor.execute(sql)
        cursor.close()
        conn.commit()
        resultDataList.append(pre_data[0][0])
        resultDataList.append(pre_data[0][1])
        resultDataList.append(pre_data[0][2])
        resultDataList.append(pre_data[0][3])

    result=[]
    tank_dict = {}
    for i, val in enumerate(resultDataList):
        if i % 4 == 0:
            if tank_dict:
                result.append(tank_dict)
                tank_dict = {}
            tank_dict['DO_PRE'] = val
        elif i % 4 == 1:
            tank_dict['PH_PRE'] = val
        elif i % 4 == 2:
            tank_dict['TEMP_PRE'] = val
        elif i % 4 == 3:
            tank_dict['SALT_PRE'] = val

    result.append(tank_dict)
    for i in range(len(result)):
        accListSet[i].update(result[i])
    print(result)


    for d in accListSet:
        d['RECORD_DATE'] = d['RECORD_DATE'].isoformat()
        d['DO'] = str(d['DO'])
        d['PH'] = str(d['PH'])
        d['TEMP'] = str(d['TEMP'])
        d['SALT'] = str(d['SALT'])
        d['DO_PRE'] = str(d['DO_PRE'])
        d['PH_PRE'] = str(d['PH_PRE'])
        d['SALT_PRE'] = str(d['SALT_PRE'])
        d['TEMP_PRE'] = str(d['TEMP_PRE'])
    
    tempData = {"type":"insertData","value":accListSet}
    jsonData = json.dumps(tempData)
    return jsonData

In [27]:
# 6시간 후 예측하기위한 함수 정의

def preData(dateTime, tankName):
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = """SELECT *
                 FROM (
                 SELECT *, ROW_NUMBER() OVER (PARTITION BY TANK_ID ORDER BY RECORD_DATE) AS row_num
                 FROM TANK_DATA
                 WHERE RECORD_DATE 
                        IN ('{dateTime}', 
                            DATE_SUB('{dateTime}', INTERVAL 6 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 12 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 18 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 24 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 30 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 36 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 42 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 48 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 54 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 60 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 66 HOUR))
                 ) AS t"""
        sql = sql.format(dateTime=dateTime)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    accListSet
    new_list = [[[elem[2:-2] for elem in subsublist] for subsublist in sublist] for sublist in data_list_of_tankList]
    for i in range(len(new_list)):
        for j in range(len(new_list[0])):
            new_list[i][j] = new_list[i][j][:12]
    scaler = MinMaxScaler()
    new_pre_data = [] # 예측후에 실제값과 예측값 및 측정 시간을 담을 배열 
    x = []
    for i, tank in enumerate(new_list):
        x.append(tank[0])
        scaler_data = scaler.fit_transform(x[i]) # 독립변수 스케일링
        reshaped_data = scaler_data.reshape(-1,12,4) # shape 변경
        pre = model_test.predict(reshaped_data) # 모델 예측              
        reshaped_array = np.tile(pre, (1, 4)) # shape 재설정
        pre_data = scaler.inverse_transform(reshaped_array)  # 실제 예측값 shape = (1,4) // DO, PH, TEMP, SALT
        print([tankList[i]])
        new_pre_data.append(np.concatenate((np.array([[tankList[i]]]), pre_data), axis=1))
    new_pre_data
    preDataDictList = []
    for row in new_pre_data:
        my_dict = {
            "TANK_ID": row[0][0],
            "DO": float(row[0][1]),
            "PH": float(row[0][2]),
            "TEMP": float(row[0][3]),
            "SALT": float(row[0][4])
        }
        preDataDictList.append(my_dict)
    
    class CustomJSONEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime):
                return obj.isoformat()
            elif isinstance(obj, (int, float)):
                return str(obj)
            else:
                return super().default(obj)
    json_data = json.dumps(preDataDictList, cls=CustomJSONEncoder)
    tempData = {"type":"predictData","value":json_data}
    jsonData = json.dumps(tempData)
    print(jsonData)
    return jsonData


In [28]:
# 10분단위 예측값 insert 할때, 같은 시간대에서 6시간후를 예측하기 위해 같은 시간대와, 수조이름을 뽑아내기 위한 함수 정의
def predictData():
    global count
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},13"
        sql = sql.format(tank_id=tank_name, count=count)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    print(accListSet[0]['RECORD_DATE'])
    return preData(accListSet[0]['RECORD_DATE'], accListSet[0]['TANK_ID']) ## 독립변수를 집어넣고 데이터 예측하기!! 
     

    

In [29]:
# 이상치 점검 함수
def insertDangerData():
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},1"
        sql = sql.format(tank_id=tank_name, count=count+11)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    # 이상치 를 담을 리스트
    dangerValueList = []


    for i in testList:
    #     print(i[0])
        # 이상치 비교 하기
        if(i[0]['TEMP']<23 or i[0]['TEMP']>32):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['TEMP'],
                'TYPE': list(i[0].keys())[2],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['DO']<4 or i[0]['DO']>10):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['DO'],
                'TYPE': list(i[0].keys())[0],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['PH']<5.5 or i[0]['PH']>8.5):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['PH'],
                'TYPE': list(i[0].keys())[1],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['SALT']<0.05 or i[0]['SALT']>5.34):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['SALT'],
                'TYPE': list(i[0].keys())[3],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
    dangerValueList[0]['VALUE']
    # 이상치 점검해서 리스트에 담음!!
    # 만약 이상치 (dangerValueList)가 null이 아닐경우 이상치를 DB(테이블 : Alarm) 에 넣기


    for dict_ in dangerValueList:
        dict_['RECORD_DATE'] = dict_['RECORD_DATE'].strftime('%Y-%m-%d %H:%M:%S')

    # convert list to JSON
    json_str = json.dumps(dangerValueList)

    if dangerValueList is not None:
        for i in dangerValueList:
            cursor = conn.cursor()
            sql = f"""insert into ALARM 
                            VALUES (NULL,
                                    '{i['TANK_ID']}',
                                    '{i['VALUE']}',
                                    '{i['RECORD_DATE']}',
                                    'insogae',
                                    '{i['TYPE']}')

                  """
            cursor.execute(sql)
            cursor.close()
        conn.commit()
        # convert datetime objects to string format
        json_str
        tempData = {"type":"dangerData","value":json_str}
        jsonData = json.dumps(tempData)
        return jsonData
    # dangerValueList가 null일경우 None 반환
    else:
        return None

In [30]:
conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')

In [ ]:
import asyncio
import websockets
import nest_asyncio
nest_asyncio.apply()


async def server(websocket, path):
    while True:
        data = await websocket.recv()
        print(data)
        response = "Hello, client!"
        task = asyncio.create_task(sendData(websocket))
        await task
        
async def sendData(websocket):
    while True:
#         test = {"type":"test","value":'테스트'} # 여러 json을 보내기 위한 테스트
#         jsonTest = json.dumps(test) 
        preData6hour = predictData() # 예측값을 통해서 6시간뒤 보여주기위한 데이터 내보내기
        dangerData = insertDangerData() # 이상치를 DB에 insert 함과 동시에 클라이언트로 데이터 보내기 (토스트에 활용)
        updateData = insertData() # 10분단위 예측값을 DB에 insert 함과 동시에 클라이언트로 데이터 보내기 (대시보드, 시각화)
        # 만약 이상치데이터가 None이 아닐경우에만 데이터 보내기
        if dangerData is not None:
            await websocket.send(dangerData)
        await websocket.send(preData6hour)
        await websocket.send(updateData)
#         await websocket.send(jsonTest)
        await asyncio.sleep(3)
        
async def main():
    async with websockets.serve(server, 'localhost', 8123):
        await asyncio.Future()  # 서버가 종료되지 않도록 무한대기

asyncio.run(main())

파이썬서버 연결!! 클라이언트: predictDataHandling.js
2021-09-28 09:50:00
1/1 [==============================] - 0s 436ms/step
['WT11']
1/1 [==============================] - 0s 23ms/step
['WT12']
1/1 [==============================] - 0s 19ms/step
['WT13']
1/1 [==============================] - 0s 17ms/step
['WT14']
1/1 [==============================] - 0s 18ms/step
['WT21']
1/1 [==============================] - 0s 18ms/step
['WT22']
1/1 [==============================] - 0s 17ms/step
['WT23']
1/1 [==============================] - 0s 17ms/step
['WT31']
{"type": "predictData", "value": "[{\"TANK_ID\": \"WT11\", \"DO\": 13.633365, \"PH\": 6.5884686, \"TEMP\": 27.602072, \"SALT\": 0.09673181}, {\"TANK_ID\": \"WT12\", \"DO\": 13.633365, \"PH\": 6.5884686, \"TEMP\": 27.602072, \"SALT\": 0.09673181}, {\"TANK_ID\": \"WT13\", \"DO\": 13.633365, \"PH\": 6.5884686, \"TEMP\": 27.602072, \"SALT\": 0.09673181}, {\"TANK_ID\": \"WT14\", \"DO\": 13.633365, \"PH\": 6.5884686, \"TEMP\": 27.602072, \"SALT\": 0.09

connection handler failed
Traceback (most recent call last):
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 1175, in _ws_handler
    return await cast(
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 13, in server
    await task
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 328, in __wakeup
    future.result()
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 201, in result
    raise self._exception
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 25, in sendData
    await websocket.send(preData6hour)
  F

[{'DO_PRE': 14.137704, 'PH_PRE': 6.563984, 'TEMP_PRE': 27.699081, 'SALT_PRE': 0.09244088}, {'DO_PRE': 9.112303, 'PH_PRE': 6.5692167, 'TEMP_PRE': 27.81347, 'SALT_PRE': 0.09195571}, {'DO_PRE': 9.383727, 'PH_PRE': 6.563593, 'TEMP_PRE': 27.891418, 'SALT_PRE': 0.09593134}, {'DO_PRE': 8.899719, 'PH_PRE': 6.5883636, 'TEMP_PRE': 27.705885, 'SALT_PRE': 0.091682605}, {'DO_PRE': 9.233132, 'PH_PRE': 6.7208357, 'TEMP_PRE': 27.62983, 'SALT_PRE': 0.43291834}, {'DO_PRE': 10.908272, 'PH_PRE': 7.129737, 'TEMP_PRE': 24.33574, 'SALT_PRE': 0.4378556}, {'DO_PRE': 11.303291, 'PH_PRE': 7.0690217, 'TEMP_PRE': 20.78364, 'SALT_PRE': 0.38338247}, {'DO_PRE': 6.596285, 'PH_PRE': 11.547835, 'TEMP_PRE': 26.668232, 'SALT_PRE': 3.363503}]
파이썬서버 연결!! 클라이언트: predictDataHandling.js
2021-09-28 11:50:00
1/1 [==============================] - 0s 16ms/step
['WT11']
1/1 [==============================] - 0s 17ms/step
['WT12']
1/1 [==============================] - 0s 16ms/step
['WT13']
1/1 [==============================] - 0s

connection handler failed
Traceback (most recent call last):
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 1175, in _ws_handler
    return await cast(
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 13, in server
    await task
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 328, in __wakeup
    future.result()
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 201, in result
    raise self._exception
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 25, in sendData
    await websocket.send(preData6hour)
  F

[{'DO_PRE': 14.562946, 'PH_PRE': 6.575299, 'TEMP_PRE': 27.719696, 'SALT_PRE': 0.092747994}, {'DO_PRE': 9.135124, 'PH_PRE': 6.5812883, 'TEMP_PRE': 27.833746, 'SALT_PRE': 0.09211079}, {'DO_PRE': 11.849392, 'PH_PRE': 6.5831056, 'TEMP_PRE': 27.940224, 'SALT_PRE': 0.096025504}, {'DO_PRE': 8.869222, 'PH_PRE': 6.5877495, 'TEMP_PRE': 27.710688, 'SALT_PRE': 0.09168291}, {'DO_PRE': 9.195583, 'PH_PRE': 6.7192955, 'TEMP_PRE': 27.658651, 'SALT_PRE': 0.43267742}, {'DO_PRE': 10.916773, 'PH_PRE': 7.101814, 'TEMP_PRE': 24.349749, 'SALT_PRE': 0.43788707}, {'DO_PRE': 11.311013, 'PH_PRE': 7.0705004, 'TEMP_PRE': 20.84088, 'SALT_PRE': 0.38393962}, {'DO_PRE': 6.6336102, 'PH_PRE': 11.565903, 'TEMP_PRE': 26.66881, 'SALT_PRE': 3.3810751}]
파이썬서버 연결!! 클라이언트: predictDataHandling.js
2021-09-28 12:10:00
1/1 [==============================] - 0s 22ms/step
['WT11']
1/1 [==============================] - 0s 20ms/step
['WT12']
1/1 [==============================] - 0s 19ms/step
['WT13']
1/1 [============================

...

[{'DO_PRE': 14.389703, 'PH_PRE': 6.612977, 'TEMP_PRE': 27.815266, 'SALT_PRE': 0.092278205}, {'DO_PRE': 9.043983, 'PH_PRE': 6.5664525, 'TEMP_PRE': 27.829535, 'SALT_PRE': 0.09147635}, {'DO_PRE': 11.923714, 'PH_PRE': 6.602573, 'TEMP_PRE': 27.927244, 'SALT_PRE': 0.09549384}, {'DO_PRE': 11.191504, 'PH_PRE': 6.607371, 'TEMP_PRE': 27.834505, 'SALT_PRE': 0.091489}, {'DO_PRE': 9.462272, 'PH_PRE': 6.770478, 'TEMP_PRE': 28.122095, 'SALT_PRE': 0.38828856}, {'DO_PRE': 10.504606, 'PH_PRE': 7.02355, 'TEMP_PRE': 24.316193, 'SALT_PRE': 0.3849471}, {'DO_PRE': 11.112003, 'PH_PRE': 7.0123997, 'TEMP_PRE': 20.731026, 'SALT_PRE': 0.32343546}, {'DO_PRE': 6.404517, 'PH_PRE': 9.16297, 'TEMP_PRE': 26.614103, 'SALT_PRE': 3.3832886}]
파이썬웹소켓연결됨!, 클라이언트: main.js
2021-09-28 20:10:00
1/1 [==============================] - 0s 17ms/step
['WT11']
1/1 [==============================] - 0s 17ms/step
['WT12']
1/1 [==============================] - 0s 18ms/step
['WT13']
1/1 [==============================] - 0s 17ms/step
['W

connection handler failed
Traceback (most recent call last):
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 1175, in _ws_handler
    return await cast(
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 13, in server
    await task
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 328, in __wakeup
    future.result()
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 201, in result
    raise self._exception
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 25, in sendData
    await websocket.send(preData6hour)
  F

[{'DO_PRE': 13.489991, 'PH_PRE': 6.603133, 'TEMP_PRE': 27.804962, 'SALT_PRE': 0.09217562}, {'DO_PRE': 8.972306, 'PH_PRE': 6.562575, 'TEMP_PRE': 27.846073, 'SALT_PRE': 0.09147633}, {'DO_PRE': 14.229417, 'PH_PRE': 6.6106224, 'TEMP_PRE': 27.936653, 'SALT_PRE': 0.095581055}, {'DO_PRE': 11.36314, 'PH_PRE': 6.6064568, 'TEMP_PRE': 27.835835, 'SALT_PRE': 0.0914434}, {'DO_PRE': 9.498035, 'PH_PRE': 6.768793, 'TEMP_PRE': 28.30401, 'SALT_PRE': 0.38285568}, {'DO_PRE': 10.561601, 'PH_PRE': 7.0282364, 'TEMP_PRE': 24.274097, 'SALT_PRE': 0.37561205}, {'DO_PRE': 11.151214, 'PH_PRE': 7.028219, 'TEMP_PRE': 20.882309, 'SALT_PRE': 0.3308558}, {'DO_PRE': 6.4271865, 'PH_PRE': 9.24557, 'TEMP_PRE': 26.632639, 'SALT_PRE': 3.3803546}]
파이썬서버 연결!! 클라이언트: predictDataHandling.js
2021-09-28 20:30:00
1/1 [==============================] - 0s 20ms/step
['WT11']
1/1 [==============================] - 0s 24ms/step
['WT12']
1/1 [==============================] - 0s 20ms/step
['WT13']
1/1 [==============================] - 

connection handler failed
Traceback (most recent call last):
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 236, in handler
    await self.ws_handler(self)
  File "C:\Users\RYU\anaconda3\lib\site-packages\websockets\legacy\server.py", line 1175, in _ws_handler
    return await cast(
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 13, in server
    await task
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 284, in __await__
    yield self  # This tells Task to wait for completion.
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 328, in __wakeup
    future.result()
  File "C:\Users\RYU\anaconda3\lib\asyncio\futures.py", line 201, in result
    raise self._exception
  File "C:\Users\RYU\anaconda3\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\RYU\AppData\Local\Temp\ipykernel_10140\3561661801.py", line 25, in sendData
    await websocket.send(preData6hour)
  F

[{'DO_PRE': 12.084661, 'PH_PRE': 6.5171347, 'TEMP_PRE': 27.74895, 'SALT_PRE': 0.09183105}, {'DO_PRE': 13.944599, 'PH_PRE': 6.572939, 'TEMP_PRE': 27.926825, 'SALT_PRE': 0.091799885}, {'DO_PRE': 11.809015, 'PH_PRE': 6.5631576, 'TEMP_PRE': 27.913614, 'SALT_PRE': 0.09536689}, {'DO_PRE': 14.224078, 'PH_PRE': 6.60089, 'TEMP_PRE': 27.83696, 'SALT_PRE': 0.09140597}, {'DO_PRE': 9.414443, 'PH_PRE': 6.74857, 'TEMP_PRE': 27.857706, 'SALT_PRE': 0.37866372}, {'DO_PRE': 10.738627, 'PH_PRE': 7.0552897, 'TEMP_PRE': 24.266157, 'SALT_PRE': 0.36470702}, {'DO_PRE': 11.15676, 'PH_PRE': 7.0279803, 'TEMP_PRE': 20.89103, 'SALT_PRE': 0.3225336}, {'DO_PRE': 6.4427896, 'PH_PRE': 11.628275, 'TEMP_PRE': 26.550156, 'SALT_PRE': 3.3749661}]


In [ ]:
# # 테스트 
# data_list_of_tankList = []
# testList = []
# for tank_name in tankList:
#     cursor = conn.cursor(DictCursor)
#     sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},13"
#     sql = sql.format(tank_id=tank_name, count=count)
#     cursor.execute(sql)
#     rows = cursor.fetchall()
#     for row in rows:
#         for key, value in row.items():
#             if isinstance(value, Decimal):
#                 row[key] = float(str(value))
#     testList.append(rows)
#     data_list_of_tankList.append([[list(row.values()) for row in rows]])
#     cursor.close()
# accListSet = []

# for i in testList:
#     accListSet.append(i[12])
# accListSet
# new_list = [[[elem[:-2] for elem in subsublist] for subsublist in sublist] for sublist in data_list_of_tankList]

# for i in range(len(new_list)):
#     for j in range(len(new_list[0])):
#         new_list[i][j] = new_list[i][j][:12]
# new_list
# #     count = count+1
# #     x = []
# #     scaler = MinMaxScaler()
# #     resultDataList = [] # 예측후에 실제값과 예측값 및 측정 시간을 담을 배열 
# #     for i, tank in enumerate(new_list):
# #         x.append(tank[0])
# #         scaler_data = scaler.fit_transform(x[i]) # 독립변수 스케일링
# #         reshaped_data = scaler_data.reshape(-1,12,4) # shape 변경
# #         pre = model_test.predict(reshaped_data) # 모델 예측
# #         reshaped_array = np.tile(pre, (1, 4)) # shape 재설정
# #         pre_data = scaler.inverse_transform(reshaped_array)  # 실제 예측값 shape = (1,4) // DO, PH, TEMP, SALT
# #         cursor = conn.cursor()
# #         sql = f"""insert into TANK_DATA_copy 
# #                     VALUES (NULL,'{tankList[i]}',{pre_data[0][0]},{pre_data[0][1]},{pre_data[0][2]},{pre_data[0][3]},
# #                     (SELECT max(record_date)
# #                     FROM    (SELECT a.*,
# #                                 ROW_NUMBER() OVER (PARTITION BY a.TANK_ID ORDER BY a.RECORD_DATE desc )  AS row_num 
# #                             from TANK_DATA_copy a ,TANK b
# #                             where a.TANK_ID = b.TANK_ID
# #                             and b.COMPANY_ID = 'insogae') sub 
# #                     where sub.row_num <= 1
# #                     and sub.TANK_ID = '{tankList[i]}'
# #                     order by sub.TANK_ID,sub.RECORD_DATE)
# #                     + INTERVAL 10 MINUTE)"""
# #         cursor.execute(sql)
# #         cursor.close()
# #         conn.commit()
# #         resultDataList.append(pre_data[0][0])
# #         resultDataList.append(pre_data[0][1])
# #         resultDataList.append(pre_data[0][2])
# #         resultDataList.append(pre_data[0][3])

# #     result=[]
# #     tank_dict = {}
# #     for i, val in enumerate(resultDataList):
# #         if i % 4 == 0:
# #             if tank_dict:
# #                 result.append(tank_dict)
# #                 tank_dict = {}
# #             tank_dict['DO_PRE'] = val
# #         elif i % 4 == 1:
# #             tank_dict['PH_PRE'] = val
# #         elif i % 4 == 2:
# #             tank_dict['TEMP_PRE'] = val
# #         elif i % 4 == 3:
# #             tank_dict['SALT_PRE'] = val

# #     result.append(tank_dict)
# #     for i in range(len(result)):
# #         accListSet[i].update(result[i])
# #     print(result)


# #     for d in accListSet:
# #         d['RECORD_DATE'] = d['RECORD_DATE'].isoformat()
# #         d['DO'] = str(d['DO'])
# #         d['PH'] = str(d['PH'])
# #         d['TEMP'] = str(d['TEMP'])
# #         d['SALT'] = str(d['SALT'])
# #         d['DO_PRE'] = str(d['DO_PRE'])
# #         d['PH_PRE'] = str(d['PH_PRE'])
# #         d['SALT_PRE'] = str(d['SALT_PRE'])
# #         d['TEMP_PRE'] = str(d['TEMP_PRE'])
# # #     accListSet = json.dumps(accListSet)
# #     tempData = {"type":"insertData","value":accListSet}
# #     jsonData = json.dumps(tempData)

In [ ]:
conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')

In [ ]:
# # 이상치 점검 함수
# def insertDangerData():
#     global count
#     data_list_of_tankList = []
#     testList = []
#     for tank_name in tankList:
#         cursor = conn.cursor(DictCursor)
#         sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},1"
#         sql = sql.format(tank_id=tank_name, count=count+11)
#         cursor.execute(sql)
#         rows = cursor.fetchall()
#         for row in rows:
#             for key, value in row.items():
#                 if isinstance(value, Decimal):
#                     row[key] = float(str(value))
#         testList.append(rows)
#         data_list_of_tankList.append([[list(row.values()) for row in rows]])
#         cursor.close()
#     testList
#     # 이상치 를 담을 리스트
#     dangerValueList = []
#     for i in testList:
#     #     print(i[0])
#         # 이상치 비교 하기
#         if(i[0]['TEMP']<23 or i[0]['TEMP']>32):
#             dangerDict ={
#                 'TANK_ID' : i[0]['TANK_ID'],
#                 'VALUE': +i[0]['TEMP'],
#                 'TYPE': [key for key, value in i if value == i[0]['TEMP']][0]
#             }
#             dangerValueList.append(dangerDict)
#         if(i[0]['DO']<4 or i[0]['DO']>10):
#             dangerDict ={
#                 'TANK_ID' : i[0]['TANK_ID'],
#                 'VALUE': i[0]['DO'],
#                 'TYPE': [key for key, value in i if value == i[0]['DO']][0]
#             }
#             dangerValueList.append(dangerDict)
#         if(i[0]['PH']<5.5 or i[0]['PH']>8.5):
#             dangerDict ={
#                 'TANK_ID' : i[0]['TANK_ID'],
#                 'VALUE': i[0]['PH'],
#                 'TYPE': [key for key, value in i if value == i[0]['PH']][0]
#             }
#             dangerValueList.append(dangerDict)
#         if(i[0]['SALT']<0.1 or i[0]['SALT']>0.34):
#             dangerDict ={
#                 'TANK_ID' : i[0]['TANK_ID'],
#                 'VALUE': i[0]['SALT'],
#                 'TYPE': [key for key, value in i if value == i[0]['SALT']][0]
#             }
#             dangerValueList.append(dangerDict)
    
#     # 이상치 점검해서 리스트에 담음!!
#     # 만약 이상치 (dangerValueList)가 null이 아닐경우 이상치를 DB(테이블 : Alarm) 에 넣기
#     if dangerValueList is not None:
#         for i in dangerValueList:
#             cursor = conn.cursor()
#             sql = f"""insert into ALARM 
#                             VALUES (NULL,
#                                     '{i[0]['VALUE']}',
#                                     '{i[0]['RECORD_DATE']}',
#                                     'insogae',
#                                     '{i[0]['TYPE']}',

#                   """
#             cursor.execute(sql)
#             cursor.close()
#             conn.commit()
# #         jsonData = json.dumps(dangerValueList)
#         tempData = {"type":"dangerData","value":dangerValueList}
#         jsonData = json.dumps(tempData)
#         return jsonData
#     # dangerValueList가 null일경우 None 반환
#     else:
#         return None;
#         #     sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count+12},1"
#         #     sql = sql.format(tank_id=tank_name, count=count)
#     #         cursor.execute(sql)
#     #         rows = cursor.fetchall()
#     #         for row in rows:
#     #             for key, value in row.items():
#     #                 if isinstance(value, Decimal):
#     #                     row[key] = float(str(value))
#     #         testList.append(rows)
#     #         data_list_of_tankList.append([[list(row.values()) for row in rows]])
#     #         cursor.close()
